In [ ]:
# time_constraint = 15000
# path_raw = "ID: 0->ID: 3->ID: 8->ID: 10->ID: 16->ID: 9->ID: 6->ID: 2->ID: 1->ID: 4->ID: 17->ID: 26->ID: 35->ID: 37->ID: 34->ID: 28->ID: 12->ID: 22->ID: 25->ID: 19->ID: 18->ID: 20->ID: 23->ID: 36->ID: 21->ID: 13->ID: 29->ID: 15->ID: 14->ID: 41->ID: 42->ID: 38->ID: 5->ID: 11->ID: 7->ID: 0->"
# legend_placement = [52.532883, 13.339717]

time_constraint = 30000
path_raw = "ID: 0->ID: 3->ID: 8->ID: 10->ID: 16->ID: 9->ID: 6->ID: 2->ID: 1->ID: 4->ID: 17->ID: 26->ID: 35->ID: 37->ID: 34->ID: 28->ID: 12->ID: 22->ID: 25->ID: 19->ID: 18->ID: 20->ID: 23->ID: 36->ID: 21->ID: 13->ID: 29->ID: 15->ID: 14->ID: 41->ID: 42->ID: 38->ID: 5->ID: 11->ID: 7->ID: 30->ID: 24->ID: 32->ID: 33->ID: 27->ID: 31->ID: 40->ID: 46->ID: 45->ID: 48->ID: 56->ID: 52->ID: 62->ID: 58->ID: 68->ID: 65->ID: 73->ID: 50->ID: 51->ID: 94->ID: 0->"
legend_placement = [52.706653, 12.874888]

In [ ]:
import datetime
import locale
locale.setlocale(locale.LC_ALL, '')

In [ ]:
extra_html = ""

In [ ]:
from context import openroute

openroute.api.go_offline()
local_ready = openroute.check_status()
print("[{}] localhost".format(' OK ' if local_ready else 'FAIL'))

openroute.api.go_online()
azure_ready = openroute.check_status()
print("[{}] azure".format(' OK ' if azure_ready else 'FAIL'))

openroute.api.toggle_online(not local_ready)

print('Using: {}'.format('azure' if openroute.api.is_online() else 'localhost'))

In [ ]:
import json

with open("data/matrix500.json", 'r') as infile:
    data = json.load(infile)

geocaches = [[lat, lon] for lon, lat in data['metadata']['query']['locations']]
cost_matrix = data['durations']

In [ ]:
path = [int(val) for val in list(filter(None, path_raw.replace('ID: ', '').split('->')))]

In [ ]:
time_constraint = 30000
legend_placement = [52.522467, 13.289700]

ant_count = 409
iterations = 55
path = [0, 3, 8, 10, 16, 14, 15, 13, 6, 2, 1, 4, 12, 22, 25, 19, 18, 20, 23, 21, 36, 43, 39, 44, 54, 57, 69, 96, 108, 130, 132, 131, 133, 135, 137, 151, 147, 146, 145, 139, 141, 144, 155, 165, 182, 191, 193, 202, 188, 197, 209, 205, 166, 149, 134, 140, 138, 121, 117, 112, 109, 95, 84, 82, 81, 78, 80, 66, 61, 49, 47, 40, 32, 33, 31, 27, 26, 35, 37, 34, 28, 17, 0]

extra_html = f"""
    <tr>
        <td>Ants:</td><td align=right>{ant_count:n}</td>
    </tr>
    <tr>
        <td>Iterations:</td><td align=right>{iterations:n}</td>
    </tr>
"""

In [ ]:
print(path)

In [ ]:
import math

cost = 0

for index in range(len(path)-1):
    from_id = path[index]
    to_id = path[index+1]
    cost += cost_matrix[from_id][to_id]

cost = math.ceil(cost)

In [ ]:
import folium
from algo_results import create_marker, create_circle_marker, fit_locations, style_function, create_legend

In [ ]:
import math
max_geocaches = int(math.ceil((max(path) + 1) / 10.0)) * 10

In [ ]:
map = folium.Map(tiles=None, location=[52.498340, 13.358513], zoom_start=14, zoom_control=False)
folium.TileLayer(tiles='Stamen Toner', opacity=0.15).add_to(map)
pass

In [ ]:
from tqdm import tqdm

optimal_locations = [[geocaches[id][1], geocaches[id][0]] for id in path]

step = 10
max_steps = math.ceil(len(optimal_locations) / step)

In [ ]:
import colorsys
import seaborn as sns

color_palette = ['#%02x%02x%02x' % (int(color[0]*255), int(color[1]*255), int(color[2]*255)) for color in sns.color_palette("hls", max_steps)]

color_palette_html = f"""
    <table class="color-palette"><tr>{''.join([f'<td style="color: {color};"><div>&nbsp;</div></td>' for color in color_palette])}</tr></table>
"""

color_palette

In [ ]:
optimal_routes = []

for index in tqdm(range(max_steps)):
    start = index * step
    request = {
        'coordinates': optimal_locations[start:start+step+1],
        'geometry': True,
        'instructions': False
    }
    optimal_route = openroute.request_directions(request, format_out='/geojson')

    if optimal_route:
        folium.features.GeoJson(data=optimal_route,
                                name='Optimal route',
                                style_function=style_function(color_palette[index]),
                                overlay=True).add_to(map)
        optimal_routes.append(optimal_route)

In [ ]:
use_penultimate = path[0] == path[-1]
start_node = path[0]
last_node = path[-2] if use_penultimate else path[-1]
visited_nodes = len(path) - 1 if use_penultimate else len(path)

for index, coords in enumerate(geocaches[:max_geocaches]):
    color = '#787c99'
    if index == start_node: # start
        color = '#41a6b5'
    elif index == last_node: # last node before start
        color = '#9d7cd8'
    elif index in path: # other visited
        color = '#3c7da2'
    create_marker(path, coords, index, color, radius=8).add_to(map)

In [ ]:
def sum_duration(routes: list):
    sum = 0.0
    for route in routes:
        sum += route['features'][0]['properties']['summary']['duration']
    return sum

optimal_duration = sum_duration(optimal_routes)
cost_str = str(datetime.timedelta(seconds=round(cost)))
actual_str = str(datetime.timedelta(seconds=round(optimal_duration)))

time_constraint_html = ''
if time_constraint:
    time_constraint_html = f"""
        <tr>
            <td>Time constraint:</td>
            <td align=right>{str(datetime.timedelta(seconds=time_constraint))}</td>
        </tr>
    """

legend = create_legend(legend_placement, html=f"""
    {color_palette_html}
    <table width=100%>
        {extra_html}
        {time_constraint_html}
        <tr>
            <td>Duration:</td>
            <td align=right>{cost_str}</td>
        </tr>
        <tr>
            <td>Cost:</td>
            <td align=right>{cost}</td>
        </tr>
        <tr>
            <td>Visited nodes:</td>
            <td align=right>{visited_nodes}</td>
        </tr>
    </table>
    <div class="marker-description">
        <span class="start">Start node</span>
        <span class="last">{"Penultimate" if use_penultimate else "Last"} node</span>
    </div>
""")

legend.add_to(map)

(cost_str, actual_str)

In [ ]:
map